# Chapter 8

## MNIST example but using convolutional networks

In [ ]:
from tensorflow import keras
from keras import layers

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8824 - loss: 0.3778
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9854 - loss: 0.0463
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9899 - loss: 0.0336
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9924 - loss: 0.0228
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.9948 - loss: 0.0170


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9879 - loss: 0.0406
Test accuracy: 0.991


## Dogs and cats dataset

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

od.download('https://www.kaggle.com/c/dogs-vs-cats')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: iagobermudez
Your Kaggle Key: ··········


100%|██████████| 812M/812M [00:39<00:00, 21.4MB/s]



Extracting archive ./dogs-vs-cats/dogs-vs-cats.zip to ./dogs-vs-cats


In [18]:
%cd /content/dogs-vs-cats

# Unzip the files quietly
!unzip -qq test1.zip
!unzip -qq train.zip

/content/dogs-vs-cats


In [19]:
%cd /content
!rm -r test1 train

/content


In [21]:
!rm -r cats_vs_dogs_small

In [22]:
import os, shutil, pathlib

original_dir = pathlib.Path("dogs-vs-cats/train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [33]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [37]:
from tensorflow import keras
from keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

### Data preprocessing

In [38]:
from keras.utils import image_dataset_from_directory
import pathlib

new_base_dir = pathlib.Path("cats_vs_dogs_small")
train_dataset = image_dataset_from_directory(
  new_base_dir / 'train',
  image_size=(180, 180),
  batch_size=32
)

test_dataset = image_dataset_from_directory(
  new_base_dir / 'test',
  image_size=(180, 180),
  batch_size=32
)

validation_dataset = image_dataset_from_directory(
  new_base_dir / 'validation',
  image_size=(180, 180),
  batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [39]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/100
63/63 [==============================] - 5s 47ms/step - loss: 0.7035 - accuracy: 0.4945 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 3s 44ms/step - loss: 0.6975 - accuracy: 0.5050 - val_loss: 0.6920 - val_accuracy: 0.5020
Epoch 3/100
63/63 [==============================] - 3s 44ms/step - loss: 0.6898 - accuracy: 0.5480 - val_loss: 0.6734 - val_accuracy: 0.5940
Epoch 4/100
63/63 [==============================] - 3s 43ms/step - loss: 0.6685 - accuracy: 0.6180 - val_loss: 0.7173 - val_accuracy: 0.5560
Epoch 5/100
63/63 [==============================] - 3s 43ms/step - loss: 0.6538 - accuracy: 0.6265 - val_loss: 0.6973 - val_accuracy: 0.5310
Epoch 6/100
63/63 [==============================] - 3s 43ms/step - loss: 0.6413 - accuracy: 0.6355 - val_loss: 0.8415 - val_accuracy: 0.5360
Epoch 7/100
63/63 [==============================] - 3s 43ms/step - loss: 0.6347 - accuracy: 0.6500 - val_loss: 0.7804 - val_accuracy: 0.5930
Epoch 

### Using a pretrained network

In [40]:
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3)
)
conv_base.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [44]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip('horizontal'),
    layers.RandomZoom(0.2),
    layers.RandomRotation(0.2)
  ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [48]:
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset
)

Epoch 1/50
63/63 [==============================] - 10s 151ms/step - loss: 8.3619 - accuracy: 0.9340 - val_loss: 2.9891 - val_accuracy: 0.9750
Epoch 2/50
63/63 [==============================] - 10s 152ms/step - loss: 7.6512 - accuracy: 0.9405 - val_loss: 2.8699 - val_accuracy: 0.9760
Epoch 3/50
63/63 [==============================] - 9s 148ms/step - loss: 8.0229 - accuracy: 0.9350 - val_loss: 2.4383 - val_accuracy: 0.9780
Epoch 4/50
63/63 [==============================] - 9s 145ms/step - loss: 4.3935 - accuracy: 0.9505 - val_loss: 9.7510 - val_accuracy: 0.9450
Epoch 5/50
63/63 [==============================] - 9s 144ms/step - loss: 5.4195 - accuracy: 0.9435 - val_loss: 2.3014 - val_accuracy: 0.9820
Epoch 6/50
63/63 [==============================] - 9s 143ms/step - loss: 5.2727 - accuracy: 0.9485 - val_loss: 1.7475 - val_accuracy: 0.9790
Epoch 7/50
63/63 [==============================] - 9s 143ms/step - loss: 4.3486 - accuracy: 0.9535 - val_loss: 2.0500 - val_accuracy: 0.9820
Epoc

### Fine tuning

In [53]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
  layer.trainable = False

In [57]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_dataset,
          epochs=30,
          validation_data=validation_dataset)

Epoch 1/30
63/63 [==============================] - 13s 172ms/step - loss: 0.3516 - accuracy: 0.9770 - val_loss: 0.4059 - val_accuracy: 0.9840
Epoch 2/30
63/63 [==============================] - 10s 166ms/step - loss: 0.3427 - accuracy: 0.9790 - val_loss: 0.5582 - val_accuracy: 0.9810
Epoch 3/30
63/63 [==============================] - 10s 162ms/step - loss: 0.4581 - accuracy: 0.9750 - val_loss: 1.2391 - val_accuracy: 0.9720
Epoch 4/30
63/63 [==============================] - 10s 159ms/step - loss: 0.3047 - accuracy: 0.9795 - val_loss: 0.4262 - val_accuracy: 0.9810
Epoch 5/30
63/63 [==============================] - 10s 156ms/step - loss: 0.2781 - accuracy: 0.9755 - val_loss: 0.4556 - val_accuracy: 0.9780
Epoch 6/30
63/63 [==============================] - 10s 157ms/step - loss: 0.3608 - accuracy: 0.9790 - val_loss: 0.4799 - val_accuracy: 0.9800
Epoch 7/30
63/63 [==============================] - 10s 157ms/step - loss: 0.2004 - accuracy: 0.9840 - val_loss: 0.3832 - val_accuracy: 0.9820